## Initialization

In [13]:
guy_folder = "/vol/scratch/guy"
cache_dir = guy_folder+"/cache/transformer_cache"

In [2]:
%pip install matplotlib seaborn pandas tqdm tensorboard

%cd {guy_folder}
!mkdir cache
!mkdir cache/transformer_cache
%pip install --no-cache-dir torch==1.4.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html

%cd {guy_folder}/cache/
!git clone https://github.com/huggingface/transformers.git
%pip install ./transformers
%pip install -U nlp

%cd {guy_folder}

## Not working!!!
!setenv TRANSFORMERS_CACHE /vol/scratch/guy/cache/transformer_cache
!setenv CUDA_VISIBLE_DEVICES 0

  Using cached matplotlib-3.3.2-cp38-cp38-manylinux1_x86_64.whl (11.6 MB)
  Using cached seaborn-0.11.0-py3-none-any.whl (283 kB)
  Using cached pandas-1.1.2-cp38-cp38-manylinux1_x86_64.whl (10.4 MB)
  Using cached tensorboard-2.3.0-py3-none-any.whl (6.8 MB)
  Using cached kiwisolver-1.2.0-cp38-cp38-manylinux1_x86_64.whl (92 kB)
  Using cached Pillow-7.2.0-cp38-cp38-manylinux1_x86_64.whl (2.2 MB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached numpy-1.19.2-cp38-cp38-manylinux2010_x86_64.whl (14.5 MB)
  Using cached scipy-1.5.2-cp38-cp38-manylinux1_x86_64.whl (25.7 MB)
  Using cached pytz-2020.1-py2.py3-none-any.whl (510 kB)
  Using cached Markdown-3.2.2-py3-none-any.whl (88 kB)
     |################################| 93 kB 192 kB/s eta 0:00:01
  Using cached tensorboard_plugin_wit-1.7.0-py3-none-any.whl (779 kB)
  Using cached protobuf-3.13.0-cp38-cp38-manylinux1_x86_64.whl (1.3 MB)
  Using cached Werkzeug-1.0.1-py2.py3-none-any.whl (298 kB)
  Using cached grp

In [3]:
# Might ruin things.
%pip install torchvision

     |################################| 5.9 MB 4.2 MB/s eta 0:00:01
     |################################| 748.8 MB 7.3 kB/s  eta 0:00:01    |#                               | 38.3 MB 14.8 MB/s eta 0:00:49     |#                               | 43.3 MB 14.8 MB/s eta 0:00:48     |##                              | 67.9 MB 49.0 MB/s eta 0:00:14     |###                             | 76.4 MB 49.0 MB/s eta 0:00:14     |###                             | 89.5 MB 49.0 MB/s eta 0:00:14     |#####                           | 121.8 MB 60.5 MB/s eta 0:00:11     |######                          | 162.5 MB 49.5 MB/s eta 0:00:12     |#######                         | 177.7 MB 49.5 MB/s eta 0:00:12     |#######                         | 180.0 MB 49.5 MB/s eta 0:00:12     |########                        | 201.4 MB 38.5 MB/s eta 0:00:15     |###########                     | 274.4 MB 43.4 MB/s eta 0:00:11     |###############                 | 371.3 MB 44.4 MB/s eta 0:00:09     |#################     

In [10]:
%%script false
# After login in through firefox to ImageNet
%cd {guy_folder}/data
!wget http://www.image-net.org/archive/stanford/fall11_whole.tar
# !tar -xvf 

--2020-09-20 15:58:22--  http://www.image-net.org/archive/stanford/fall11_whole.tar
Resolving www.image-net.org (www.image-net.org)... 171.64.68.16
Connecting to www.image-net.org (www.image-net.org)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2020-09-20 15:58:22 ERROR 404: Not Found.



In [440]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision.datasets import ImageNet, ImageFolder, CIFAR10
from torchvision import transforms
from transformers import AutoModel, AutoTokenizer

In [708]:
class PlainBERT(nn.Module):
    def __init__(self, n_tokens):
        super().__init__()
        bert = AutoModel.from_pretrained('distilbert-base-uncased', cache_dir = cache_dir)
        self.position_embeddings = torch.Tensor(bert.embeddings.position_embeddings(torch.arange(512)).detach().numpy())
        self.bert = bert.transformer
        self.bert.requires_grad_(False)
        self.nLayers = 6
        
    def forward(self, x):
        return self.bert.forward(x + self.position_embeddings, attn_mask = torch.ones(x.shape[0], 512),
                                head_mask = torch.ones(self.nLayers, x.shape[0], 512))

class BertVision(nn.Module):
    def __init__(self,  n_classes, img_dim):
        super().__init__()
        self.with_classifier = True
        self.n_tokens = np.prod(img_dim)
        self.top = nn.Sequential(
                                 nn.Conv2d(3, 32, 3, padding = 1 ),
                                 nn.LeakyReLU(0.2),
                                 nn.Conv2d(32, 100, 3, padding = 1),
                                 nn.LeakyReLU(0.2),
                                 nn.Conv2d(100, 200, 3, padding = 1),
                                 nn.LeakyReLU(0.2),
                                 nn.Conv2d(200, 768, 3, stride = (1, 2), padding = 1),
                                 nn.LeakyReLU(0.2)
                                )
        
        self.top.apply(self._init_top)
        self.bert = PlainBERT(n_tokens = self.n_tokens)
        self.fc = nn.Linear(768 * self.n_tokens//2, n_classes)
        self.layer_norm = nn.LayerNorm((512,))

    def toggleIntermediate(self):
        self.with_classifier = !self.with_classifier
    
    
    def _init_top(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            nn.init.kaiming_uniform_(m.weight)
            m.bias.data.fill_(0.01)
        pass
    def forward(self, x):
        x = self.top(x)
        x = x.view(x.size(0), x.size(1), -1)
        x = x.transpose(1,2)
        x = self.bert(x)
        x = torch.stack(x).squeeze(1)
        x = x.transpose(1,2).contiguous()
#         x = self.layer_norm(x)
#         x = torch.mean(x, dim = (-2,))
        x = x.view(x.size(0), -1)
        if self.with_classifier:
            x = self.fc(x)
        return x.squeeze(1)

In [442]:
train_folder = "{}/data/imagenet12/train".format(guy_folder)

train_ds = ImageFolder(train_folder,
            transform = transforms.Compose([transforms.ToTensor()]))

In [443]:
train_ds = CIFAR10("{}/data/cifar10".format(guy_folder), download = True, transform=transforms.ToTensor())

Files already downloaded and verified


In [709]:
model = BertVision(len(train_ds.classes), (32,32))

In [722]:
from collections import defaultdict

total = int(len(train_ds) * .1)

class_rep = defaultdict(int)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-6)
train_dataloader = DataLoader(train_ds, batch_size = None, shuffle = True)
pbar = tqdm(enumerate(train_dataloader), total = total)
acc_sum = 0

for i, (x,y) in pbar:
    if i >= total:
        break
    optimizer.zero_grad()
    yhat = model(x.unsqueeze(0))
    loss = criterion(yhat, torch.LongTensor([y]))
    acc_sum += yhat.argmax().item() == y
    class_rep[y] += 1
    topk = np.where(np.argsort(yhat.detach().numpy()) == y)[1][0]
    pbar.set_postfix_str("Loss: {:.2f} Acc: {:.2f} Top: {} Class: {}".format(loss.item(), 
                                                                             acc_sum/(i+1), 
                                                                             topk, class_rep[y]))
    loss.backward()
    optimizer.step()

  0%|          | 2/5000 [00:56<39:11:52, 28.23s/it, Loss: 2.85 Acc: 0.33 Top: 4 Class: 1]


KeyboardInterrupt: 

In [617]:
y  = []
z = []
for x, _  in tqdm(train_ds):
    z.append(x)
    y.append((model(x.unsqueeze(0))).detach())

  0%|          | 56/50000 [00:12<3:03:01,  4.55it/s]


KeyboardInterrupt: 

In [618]:
y = torch.stack(y)
z = torch.stack(z)

In [619]:
z.std()/y.std()

tensor(0.1974)

In [713]:
i = np.random.choice(len(train_ds))
print(F.softmax(model(train_ds[i][0].unsqueeze(0))))
print(model(train_ds[i][0].unsqueeze(0)).argmax())
print(train_ds[i][1])
t = transforms.Compose([
#     transforms.Grayscale(),
    transforms.ToPILImage(),
    transforms.Grayscale(3),
    transforms.ToTensor(),
    
    ])(train_ds[i][0])
print(F.softmax(model(t.unsqueeze(0))))
print(model(t.unsqueeze(0)).argmax())

print(F.softmax(model(((train_ds[i+1][0])).unsqueeze(0))))
print(F.softmax(model(((train_ds[i+1][0])).unsqueeze(0))).argmax())


<ipython-input-713-fe7475450bf5>:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(F.softmax(model(train_ds[i][0].unsqueeze(0))))


tensor([[0.0854, 0.0993, 0.1222, 0.1293, 0.1071, 0.0606, 0.1577, 0.1181, 0.0355,
         0.0847]], grad_fn=<SoftmaxBackward>)
tensor(6)
7


<ipython-input-713-fe7475450bf5>:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(F.softmax(model(t.unsqueeze(0))))


tensor([[0.1851, 0.1524, 0.1046, 0.0928, 0.0759, 0.0412, 0.0796, 0.0728, 0.0872,
         0.1084]], grad_fn=<SoftmaxBackward>)
tensor(0)
tensor([[0.1372, 0.2561, 0.0673, 0.0738, 0.0361, 0.0418, 0.0466, 0.0461, 0.1279,
         0.1671]], grad_fn=<SoftmaxBackward>)
tensor(1)


<ipython-input-713-fe7475450bf5>:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(F.softmax(model(((train_ds[i+1][0])).unsqueeze(0))))
<ipython-input-713-fe7475450bf5>:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(F.softmax(model(((train_ds[i+1][0])).unsqueeze(0))).argmax())


In [661]:
model.load_state_dict(state_dict,strict = False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['fc.weight', 'fc.bias'])

In [721]:
model.toggleIntermediate()
i = np.random.choice(len(train_ds))
a1 = (F.softmax(model(train_ds[i][0].unsqueeze(0))))

t = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Grayscale(3),
#     transforms.ColorJitter(10,10, 10),
#     transforms.RandomRotation(90),
    transforms.ToTensor(),
    
    ])(train_ds[i][0])
a2 = (F.softmax(model(t.unsqueeze(0))))

a3 = (F.softmax(model(((train_ds[i+1][0])).unsqueeze(0))))


print(torch.norm(a1-a2, p = 2))
print(torch.norm(a1-a3, p = 2))
print(torch.norm(a2-a3, p = 2))
model.toggleIntermediate()


<ipython-input-721-f53303e89c02>:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a1 = (F.softmax(model(train_ds[i][0].unsqueeze(0))))
<ipython-input-721-f53303e89c02>:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a2 = (F.softmax(model(t.unsqueeze(0))))


tensor(0.4107, grad_fn=<NormBackward0>)
tensor(0.6480, grad_fn=<NormBackward0>)
tensor(0.2511, grad_fn=<NormBackward0>)


<ipython-input-721-f53303e89c02>:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a3 = (F.softmax(model(((train_ds[i+1][0])).unsqueeze(0))))
